<a href="https://colab.research.google.com/github/deltorobarba/finance/blob/main/robust_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Robust Optimization**

> Accounts for uncertainty in return, volatility, and correlation estimates, optimizing for worst-case scenarios

The robust optimization approach assumes uncertainty in the estimates of the returns, aiming to generate a portfolio that performs well even under adverse conditions.

We will use the cvxpy library, which is well-suited for convex optimization problems like portfolio optimization with uncertainty.

In [1]:
!pip install cvxpy -q

In [3]:
import numpy as np
import cvxpy as cp

# Define problem data
np.random.seed(42)
n_assets = 5
expected_returns = np.array([0.12, 0.10, 0.08, 0.07, 0.05])  # Example expected returns
cov_matrix = np.array([
    [0.005, -0.002, 0.001, 0.001, 0.0],
    [-0.002, 0.004, -0.001, 0.0, 0.001],
    [0.001, -0.001, 0.003, 0.001, -0.001],
    [0.001, 0.0, 0.001, 0.004, 0.0],
    [0.0, 0.001, -0.001, 0.0, 0.002]
])  # Covariance matrix of the assets

# Uncertainty set size (this controls the level of robustness)
uncertainty_radius = 0.05  # Larger values mean more robustness

# Define optimization variables
weights = cp.Variable(n_assets)
aux_var = cp.Variable()  # Auxiliary variable to model the worst-case scenario

# Objective: Maximize worst-case returns
portfolio_return = expected_returns @ weights
portfolio_risk = cp.quad_form(weights, cov_matrix)  # Portfolio variance (risk)

# Constraints
constraints = [
    cp.norm(weights) <= aux_var,  # Auxiliary constraint for robustness
    cp.sum(weights) == 1,  # Weights sum to 1
    weights >= 0  # No short-selling
]

# Objective function: maximize the worst-case return, subject to risk penalty
risk_aversion = 0.1  # Adjust this to control risk tolerance
objective = cp.Maximize(portfolio_return - risk_aversion * portfolio_risk - uncertainty_radius * aux_var)

# Define the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal portfolio weights:")
print(weights.value)
print("Optimal expected return (robust):", portfolio_return.value)
print("Portfolio risk:", portfolio_risk.value)

Optimal portfolio weights:
[6.08090588e-01 3.35871195e-01 5.60381980e-02 1.74030711e-08
 1.54349922e-09]
Optimal expected return (robust): 0.11104104719150257
Portfolio risk: 0.0015230784341580597


Key Elements:
* **Auxiliary Variable (aux_var)**: We introduced this variable to handle the uncertainty in the optimization problem indirectly. This avoids breaking DCP rules.
* **Objective**: The term uncertainty_radius * aux_var models the worst-case effect of uncertainty, penalizing the portfolio based on the robustness requirement.
- **Robust Optimization**: Modeled using the `delta` variable, which represents the uncertainty in the expected returns. We enforce that the uncertainty in returns lies within a ball of a given radius (`uncertainty_radius`).
- **Objective**: Maximize the worst-case return considering this uncertainty, penalized by the portfolio's risk.
- **Constraints**:
  - Sum of portfolio weights equals 1.
  - Non-negative weights (no short-selling).
  - The uncertainty in returns (represented by `delta`) is bounded by the `uncertainty_radius`.

Adjustable Parameters:
- `uncertainty_radius`: Controls how robust the portfolio is to uncertainties in the expected returns. Larger values mean more conservative (robust) portfolios.
- `risk_aversion`: Balances between maximizing return and minimizing risk.

These parameters can be adjusted to observe how the portfolio shifts under different levels of uncertainty and risk tolerance.

The covariance matrix used in the example is the matrix that defines the risk (volatility) of the assets in the portfolio. This matrix is essential for portfolio optimization since it helps calculate the overall portfolio variance based on the individual asset risks and how they interact (correlate) with each other.

* The diagonal elements represent the variance of each asset. For example, asset 1 has a variance of 0.005, and asset 5 has a variance of 0.002.
* The off-diagonal elements represent the covariance between pairs of assets. For example, the covariance between asset 1 and asset 2 is -0.002, indicating that they are negatively correlated.
* A positive covariance means that two assets tend to move in the same direction, while a negative covariance means they tend to move in opposite directions.

In this case, the covariance matrix encodes how each asset in the portfolio correlates with the others, which affects the overall portfolio risk. Covariance matrices are fundamental in determining the diversification benefits (or lack thereof) when selecting portfolio weights.

In [4]:
import numpy as np
import pandas as pd

# Example covariance matrix (from the earlier example)
cov_matrix = np.array([
    [0.005, -0.002, 0.001, 0.001, 0.0],
    [-0.002, 0.004, -0.001, 0.0, 0.001],
    [0.001, -0.001, 0.003, 0.001, -0.001],
    [0.001, 0.0, 0.001, 0.004, 0.0],
    [0.0, 0.001, -0.001, 0.0, 0.002]
])

# Asset names for better readability
assets = ['Asset 1', 'Asset 2', 'Asset 3', 'Asset 4', 'Asset 5']

# Create a DataFrame for better display
cov_matrix_df = pd.DataFrame(cov_matrix, index=assets, columns=assets)

# Display the covariance matrix
import ace_tools as tools; tools.display_dataframe_to_user(name="Covariance Matrix", dataframe=cov_matrix_df)


ModuleNotFoundError: No module named 'ace_tools'

In [5]:
import numpy as np
import pandas as pd
import cvxpy as cp

# Step 1: Generate or load asset return data (using random data here for example)
np.random.seed(42)
n_assets = 5
n_obs = 1000

# Random returns data (replace this with real returns data)
returns = np.random.randn(n_obs, n_assets) * 0.01  # Simulated daily returns

# Step 2: Calculate the covariance matrix from the returns data
cov_matrix = np.cov(returns, rowvar=False)

# Step 3: Define Robust Optimization problem using the calculated covariance matrix
expected_returns = np.mean(returns, axis=0)  # Expected returns from the data
uncertainty_radius = 0.05  # Robustness parameter

# Define optimization variables
weights = cp.Variable(n_assets)
aux_var = cp.Variable()  # Auxiliary variable for robustness

# Objective: Maximize worst-case returns
portfolio_return = expected_returns @ weights
portfolio_risk = cp.quad_form(weights, cov_matrix)  # Use the calculated covariance matrix

# Constraints
constraints = [
    cp.norm(weights) <= aux_var,  # Auxiliary constraint for robustness
    cp.sum(weights) == 1,  # Weights sum to 1
    weights >= 0  # No short-selling
]

# Objective function: maximize the worst-case return, subject to risk penalty
risk_aversion = 0.1  # Adjust this to control risk tolerance
objective = cp.Maximize(portfolio_return - risk_aversion * portfolio_risk - uncertainty_radius * aux_var)

# Define the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal portfolio weights:")
print(weights.value)

# Step 4: Display the covariance matrix
cov_matrix_df = pd.DataFrame(cov_matrix, columns=[f'Asset {i+1}' for i in range(n_assets)],
                             index=[f'Asset {i+1}' for i in range(n_assets)])

import ace_tools as tools; tools.display_dataframe_to_user(name="Covariance Matrix", dataframe=cov_matrix_df)


Optimal portfolio weights:
[0.19961518 0.19796946 0.19990345 0.20008703 0.20242488]


ModuleNotFoundError: No module named 'ace_tools'